In [ ]:
#r "nuget: FSharp.Data"
#r "nuget: Plotly.NET, 2.0.0-preview.16"
#r "nuget: Plotly.NET.Interactive, 2.0.0-preview.16"
#r "nuget: XPlot.Plotly"
#r "nuget: XPlot.Plotly.Interactive"

Installed Packages FSharp.Data, 5.0.2 Plotly.NET, 2.0.0-preview.16 Plotly.NET.Interactive, 2.0.0-preview.16 XPlot.Plotly, 4.0.6 XPlot.Plotly.Interactive, 4.0.6

Loading extensions from `Plotly.NET.Interactive.dll`

Loading extensions from `XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

In [ ]:
open FSharp.Data
open Plotly.NET
open XPlot.Plotly

In [ ]:
let text = CsvFile.Load(System.IO.Path.Join([|__SOURCE_DIRECTORY__; "../data/pizza.csv"|]), separators=",", hasHeaders=true)

let x,y = [|for row in text.Rows -> (float row.[0], float row.[1])|] |> Array.unzip

In [ ]:
x.[0..4]

index,value
0,13
1,2
2,14
3,23
4,13


In [ ]:
y.[0..4]

index,value
0,33
1,16
2,32
3,51
4,27


In [ ]:
Scatter(x=x, y=y, mode = "markers")
|> Chart.Plot
|> Chart.WithXTitle "Reservations"
|> Chart.WithYTitle "Pizzas"

<!DOCTYPE html>

In [ ]:
Chart.Point(x, y, "test")
|> Chart.withXAxisStyle("Reservations")
|> Chart.withYAxisStyle("Pizzas")

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let predict (x: float[]) weight = Array.map ((*) weight) x

let average (x: float[]) = (Array.fold (+) 0.0 x) / (float x.Length)

let (.+) (x: float[]) (y: float[]) = Array.map2 (+) x y
let (.-) (x: float[]) (y: float[]) = Array.map2 (-) x y
let (.^) (x: float[]) y = Array.map (fun e -> e ** y) x 

let loss (x: float[]) (y: float[]) weight =
    average(((predict x weight) .- y) .^ 2.0)

let train x y iterations learningRate =
    let mutable w = 0.0
    for i in 0..iterations do
        let currentLoss = loss x y w
        //printfn "Iteration %4d => Loss: %.6f" i currentLoss
        if loss x y (w + learningRate) < currentLoss then
            w <- w + learningRate
        elif loss x y (w - learningRate) < currentLoss then
            w <- w - learningRate
    w

In [ ]:
let w = train x y 10000 0.01
printfn "w=%.3f" w
printfn "Prediction: x=%d => y=%.2f" 20 (predict [|20.0|] w).[0]

w=1.840
Prediction: x=20 => y=36.80


In [ ]:
let lineX = [| (Array.min x)..1.0..(Array.max y) |]
let lineY = predict lineX w

[Scatter(x=X, y=Y, name="Data", mode = "markers");
 Scatter(x=lineX, y=lineY, name="Prediction")]
|> Chart.Plot
|> Chart.WithXTitle "Reservations"
|> Chart.WithYTitle "Pizzas"

<!DOCTYPE html>

In [ ]:
let predict2 (x: float[]) weight bias = Array.map (fun e -> e*weight + bias) x

let loss2 (x: float[]) (y: float[]) weight bias =
    average(((predict2 x weight bias) .- y) .^ 2.0)

let train2 x y iterations learningRate =
    let mutable weight = 0.0
    let mutable bias = 0.0
    for i in 0..iterations do
        let currentLoss = loss2 x y weight bias
        if loss2 x y (weight + learningRate) bias < currentLoss then
            weight <- weight + learningRate
        elif loss2 x y (weight - learningRate) bias < currentLoss then
            weight <- weight - learningRate
        elif loss2 x y weight (bias + learningRate) < currentLoss then
            bias <- bias + learningRate
        elif loss2 x y weight (bias - learningRate) < currentLoss then
            bias <- bias - learningRate
    weight, bias

In [ ]:
let w, b = train2 x y 10000 0.01

printfn "w=%.3f, b=%.3f" w b
printfn "Prediction: x=%d => y=%.2f" 20 (predict2 [|20.0|] w b).[0]

w=1.100, b=12.930
Prediction: x=20 => y=34.93


In [ ]:
let lineX = [| (Array.min x)..1.0..(Array.max x) |]
let lineY = predict2 lineX w b

[Scatter(x=x, y=y, name="Data", mode = "markers");
 Scatter(x=lineX, y=lineY, name="Prediction")]
|> Chart.Plot
|> Chart.WithXTitle "Reservations"
|> Chart.WithYTitle "Pizzas"

<!DOCTYPE html>